In [1]:
%pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 581.0 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Làm sạch từng data

In [7]:
import pandas as pd
import re
import unicodedata
import string
import ast

# ==== CÁC HÀM CƠ BẢN ====

def clean_title(title: str) -> str:
    if pd.isna(title): return ''
    title = str(title).lower().strip().replace('&', 'và')
    title = title.translate(str.maketrans('', '', string.punctuation.replace('&', '').replace("'", '')))
    return re.sub(r'\s+', ' ', title).strip()

def clean_lyrics(lyrics: str) -> str:
    if pd.isna(lyrics) or not lyrics: return ''
    lyrics = str(lyrics).strip()
    lyrics = re.sub(r'\s+', ' ', lyrics)
    return lyrics[:5000]

# ==== CHUẨN HÓA & LÀM SẠCH GENRE ====

def clean_genre(genre: str) -> str:
    if not isinstance(genre, str): return ''
    genre = genre.strip()
    genre = re.sub(r'[\(\)\.]', '', genre)
    genre = re.sub(r'nhạc|nhac', '', genre, flags=re.IGNORECASE)
    genre = re.sub(r'\s+', ' ', genre).strip()
    return genre

def _strip_accents_for_key(s: str) -> str:
    if not isinstance(s, str): return ''
    s = s.strip().lower()
    s = unicodedata.normalize('NFKD', s)
    s = ''.join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r'[-–_]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def normalize_genre(genre: str) -> str:
    if not isinstance(genre, str) or not genre.strip(): return ''
    orig_clean = clean_genre(genre)
    key = _strip_accents_for_key(orig_clean)
    mapping = {
        'tre': 'Trẻ', 'tru tinh': 'Trữ Tình', 'trinh': 'Trịnh', 'vpop': 'V-Pop',
        'viet remix': 'Việt Remix', 'que huong': 'Quê Hương', 'dan ca': 'Dân Ca',
        'dan gian': 'Dân Ca', 'nuoc ngoai': 'Nước Ngoài', 'ngoai': 'Nước Ngoài',
        'ngoai han quoc': 'Ngoại Hàn Quốc', 'ngoai my': 'Ngoại Mỹ',
        'ngoai trung hoa': 'Ngoại Trung Hoa', 'ngoai phap': 'Ngoại Pháp',
        'ngoai nhat': 'Ngoại Nhật', 'ngoai nga': 'Ngoại Nga', 'ngoai duc': 'Ngoại Đức',
        'ngoai thai lan': 'Ngoại Thái Lan', 'ngoai loi viet': 'Ngoại Lời Việt',
        'hoa loi viet': 'Hoa Lời Việt', 'phap loi viet': 'Pháp Lời Việt',
        'nhat loi viet': 'Nhật Lời Việt', 'han loi viet': 'Hàn Lời Việt',
        'dan ca loi viet': 'Dân Ca Lời Việt', 'cach mang': 'Cách Mạng',
        'phat giao': 'Phật Giáo', 'edm viet': 'EDM Việt', 'rb viet': 'R&B Việt',
        'rock viet': 'Rock Việt', 'vang': 'Vàng', 'beat': 'Beat',
        'thieu nhi': 'Thiếu Nhi', 'khong loi': 'Không Lời', 'hoa tau': 'Hòa Tấu',
        'thanh ca': 'Tôn Giáo', 'cai luong': 'Cải Lương', 'phim': 'Phim',
        'tet': 'Xuân', 'giang sinh': 'Giáng Sinh', 'chill': 'Chill', 'khac': 'Khác'
    }
    if key in mapping: return mapping[key]
    for k, v in mapping.items():
        if k in key: return v
    return re.sub(r'\s+', ' ', orig_clean).strip().title()

# ==== CLEAN COMPOSER / LYRICIST (CÓ CHUYỂN GENRE) ====

def clean_person_list(persons):
    """
    Làm sạch tên và chuyển 'Nhạc ... Lời Việt' sang genres.
    Trả về tuple: (danh_sách_người_dọn_sạch, genres_chuyển_từ_person)
    """
    if not isinstance(persons, list):
        return [], []
    cleaned, transferred_genres = [], []
    trash_terms = ["nhiều nhạc sĩ", "nhiều nghệ sĩ", "nhiều ca sĩ", "various artists", "nhiều tác giả", "tác giả"]

    for p in persons:
        if not isinstance(p, str): continue
        p = re.sub(r"[^a-zA-ZÀ-ỹ\s\-\(\)\.]", "", p.strip())
        p = re.sub(r'\s+', ' ', p).strip()
        if not p: continue

        # Nếu chứa “Nhạc ... Lời Việt” => chuyển sang genre
        if re.search(r"nhạc\s*[\w\sà-ỹ]*\s*lời\s*việt", p, flags=re.IGNORECASE):
            transferred_genres.append(p.strip())
            continue
        if p.lower() in trash_terms: continue

        cleaned.append(p)

    return cleaned, transferred_genres

# ==== XỬ LÝ FILE CSV ====

def parse_list_str(x):
    if pd.isna(x): return []
    if isinstance(x, list): return x
    try:
        val = ast.literal_eval(x)
        if isinstance(val, list): return val
    except Exception:
        pass
    x = x.strip().strip("[]")
    if not x: return []
    return [i.strip().strip("'\"") for i in re.split(r'\s*,\s*', x) if i.strip()]

def clean_csv_input(path):
    df = pd.read_csv(path)
    for c in ['composers', 'lyricists', 'genres', 'urls']:
        if c in df.columns:
            df[c] = df[c].apply(parse_list_str)

    # Chuyển 'Nhạc ... Lời Việt' sang genre
    transferred_genres = [[] for _ in range(len(df))]
    for col in ['composers', 'lyricists']:
        if col not in df.columns:
            continue
        new_col = []
        for i, persons in enumerate(df[col]):
            cleaned, extra_genres = clean_person_list(persons)
            new_col.append(cleaned)
            transferred_genres[i].extend(extra_genres)
        df[col] = new_col

    # Chuẩn hóa genres
    total_added_genres = 0
    if 'genres' in df.columns:
        new_genres = []
        for i, g in enumerate(df['genres']):
            merged = g + transferred_genres[i]
            normalized = [normalize_genre(x) for x in merged if x]
            unique_norm = list(dict.fromkeys(normalized))
            total_added_genres += len(transferred_genres[i])
            new_genres.append(unique_norm)
        df['genres'] = new_genres

    # Làm sạch các trường chính
    df['title'] = df['title'].apply(lambda x: clean_title(x))
    if 'lyrics' in df.columns:
        df['lyrics'] = df['lyrics'].apply(lambda x: clean_lyrics(x))

    # === Ghi đè file gốc ===
    df.to_csv(path, index=False)

    print(f"✅ Làm sạch xong {path} ({len(df)} dòng)")
    print(f"🎵 Tổng thể loại được thêm từ composer/lyricist: {total_added_genres}")
    return df

# ==== DEMO DÙNG ====
df = clean_csv_input('N05_timbaihat_final.csv')
df.head()


✅ Làm sạch xong N05_timbaihat_final.csv (13764 dòng)
🎵 Tổng thể loại được thêm từ composer/lyricist: 1742


,title,composers,lyricists,year,genres,lyrics,urls,source,note
0,anh không phải là chú rể,[San Ji],[San Ji],2025,[V-Pop],Em giờ đã có người khác Còn anh lang thang tro...,[https://timbaihat.com/loi-bai-hat-anh-khong-p...,Việt Nam,NaN
1,đi một đường về hai hướng ost cha tôi người ở lại,[Phùng Tiến Minh],[Phùng Tiến Minh],2025,"[Phim, V-Pop]",Hoa phiêu phiêu Cây xiêu xiêu Theo chân ai về ...,[https://timbaihat.com/loi-bai-hat-di-mot-duon...,Việt Nam,NaN
2,em,"[DTAP, INUS]","[DTAP, INUS]",2025,[V-Pop],Ngắm nhìn em lặng ngồi Trong ánh dương tàn Câu...,[https://timbaihat.com/loi-bai-hat-em-dong-nhi...,Việt Nam,NaN
3,tình yêu bất diệt remix,[DK Lâm],[DK Lâm],2025,[Dance Việt],Hàng ngàn người trong nhân gian Nhưng ta chỉ s...,[https://timbaihat.com/loi-bai-hat-tinh-yeu-ba...,Việt Nam,NaN
4,khó chiều,[DTAP],[DTAP],2025,[V-Pop],Đừng nói tôi thật kiêu Khi mà ta chưa hề quen ...,[https://timbaihat.com/loi-bai-hat-kho-chieu-d...,Việt Nam,NaN


# Merge cách 1 (điều kiện title trùng + ít nhất 1 composer trùng)

In [28]:
import pandas as pd
from rapidfuzz import fuzz

# ==== HÀM HỖ TRỢ ====

def parse_list_str(x):
    """Chuyển chuỗi dạng ['A', 'B'] thành list ['A', 'B']."""
    if pd.isna(x) or not isinstance(x, str):
        return []
    x = x.strip()
    if x.startswith('[') and x.endswith(']'):
        x = x[1:-1]
    if not x.strip():
        return []
    items = [i.strip().strip("'\"") for i in x.split(',') if i.strip()]
    return items

def get_comp_key(composers):
    """Trả về frozenset để so sánh nhanh danh sách composer."""
    if not composers:
        return frozenset()
    try:
        return frozenset([c.strip().lower() for c in composers if isinstance(c, str)])
    except Exception:
        return frozenset()

def merge_unique_case_insensitive(a, b):
    """Gộp hai list chuỗi, không phân biệt hoa thường và tránh trùng."""
    merged = []
    seen = set()
    for lst in (a, b):
        if lst:
            for item in lst:
                if not isinstance(item, str):
                    continue
                key = item.lower().strip()
                if key not in seen and key != '':
                    seen.add(key)
                    merged.append(item.strip())
    return merged

def merge_rows(r1, r2, debug=False, debug_counter=[0], debug_limit=10):
    """Gộp hai dòng trùng."""
    def is_empty(x):
        if x is None:
            return True
        if isinstance(x, (list, tuple, set)):
            return len(x) == 0
        if isinstance(x, str):
            return x.strip() == ''
        try:
            return pd.isna(x)
        except Exception:
            return False

    def pick(a, b):
        return a if not is_empty(a) else b

    merged = {
        'title': pick(r1.get('title'), r2.get('title')),
        'composers': merge_unique_case_insensitive(r1.get('composers'), r2.get('composers')),
        'lyricists': merge_unique_case_insensitive(r1.get('lyricists'), r2.get('lyricists')),
        'year': pick(r1.get('year'), r2.get('year')),
        'genres': merge_unique_case_insensitive(r1.get('genres'), r2.get('genres')),
        'lyrics': pick(r1.get('lyrics'), r2.get('lyrics')),
        'urls': merge_unique_case_insensitive(r1.get('urls'), r2.get('urls')),
        'source': pick(r1.get('source'), r2.get('source')),
        'note': pick(r1.get('note'), r2.get('note')),
    }

    if debug and debug_counter[0] < debug_limit:
        debug_counter[0] += 1
        print(f"\n🧩 DEBUG MERGE #{debug_counter[0]}: '{merged['title']}'")
        print(f"  composers={merged['composers']} | lyricists={merged['lyricists']} | year={merged['year']}")
        print(f"  genres={merged['genres']} | urls={merged['urls']}")

    return merged


# ==== ĐỌC / GHI FILE ====

def read_df(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    for c in ['composers', 'lyricists', 'genres', 'urls']:
        if c in df.columns:
            df[c] = df[c].apply(parse_list_str)
    df['comp_set'] = df['composers'].apply(get_comp_key)
    print(f"📘 Đọc {path} ({len(df)} dòng)")
    return df

def write_df(df: pd.DataFrame, path: str):
    df = df.copy()
    for c in ['composers', 'lyricists', 'genres', 'urls']:
        if c in df.columns:
            df[c] = df[c].apply(str)
    df.to_csv(path, index=False)
    print(f"💾 Lưu: {path}")


# ==== GỘP DỮ LIỆU CHÍNH ====

def merge_datasets(files, output_path='merged_final.csv', debug_merge=True, debug_limit=10):
    current = read_df(files[0])
    print(f"\n🚀 Bắt đầu merge với {files[0]}")
    debug_counter = [0]
    total_merge = 0
    merged_pairs = set()

    for i, file in enumerate(files[1:], 2):
        print(f"\n🔄 Gộp file {i}/{len(files)}: {file}")
        new_df = read_df(file)
        to_add = []
        merge_count = 0
        grouped_current = current.groupby('title')

        for _, row in new_df.iterrows():
            title_key = row['title']
            same_title = grouped_current.get_group(title_key) if title_key in grouped_current.groups else None

            if same_title is None or same_title.empty:
                to_add.append(row)
                continue

            matched = False
            for idx, cand in same_title.iterrows():
                comp_overlap = bool(cand['comp_set'] & row['comp_set'])
                # chỉ cần trùng ít nhất 1 composer là merge
                if comp_overlap:
                    merged = merge_rows(cand, row, debug=debug_merge, debug_counter=debug_counter, debug_limit=debug_limit)
                    for col, val in merged.items():
                        current.at[idx, col] = val
                    current.at[idx, 'comp_set'] = get_comp_key(current.at[idx, 'composers'])
                    matched = True
                    merge_count += 1
                    merged_pairs.add(f"{cand['title']}::{','.join(cand['composers'])}")
                    break

            if not matched:
                to_add.append(row)

        if to_add:
            current = pd.concat([current, pd.DataFrame(to_add)], ignore_index=True)

        print(f"✅ Đã merge {merge_count}, thêm {len(to_add)} dòng mới.")
        total_merge += merge_count

    final = current.drop(columns=['comp_set'])
    write_df(final, output_path)
    print("\n📊 TỔNG KẾT:")
    print(f"   Tổng số bài merge: {total_merge}")
    print(f"   Tổng số bài sau merge: {len(final)}")
    return final, merged_pairs


# ==== DEMO ====

files = [
    'N01_hopamviet_final.csv',
    'N02_timbaihat_final.csv',
    'N03_loibaihat_final.csv',
    'N04_nhacvn_final.csv',
    'N04_tkaraoke_final.csv',
    'N05_timbaihat_final.csv'
]

result, merged = merge_datasets(
    files,
    output_path='merged_final.csv',
    debug_merge=True,
    debug_limit=3
)


📘 Đọc N01_hopamviet_final.csv (10422 dòng)

🚀 Bắt đầu merge với N01_hopamviet_final.csv

🔄 Gộp file 2/6: N02_timbaihat_final.csv
📘 Đọc N02_timbaihat_final.csv (18503 dòng)

🧩 DEBUG MERGE #1: '60 năm cuộc đời'
  composers=['Y Vân'] | lyricists=['Y Vân', 'Miu Lê'] | year=2015
  genres=['Trữ Tình', 'Trẻ', 'V-Pop'] | urls=['https://hopamviet.vn/chord/song/60-nam-cuoc-doi/W8IUII6C.html', 'https://timbaihat.com/loi-bai-hat-60-nam-cuoc-doi-miu-le.html']

🧩 DEBUG MERGE #2: '7 love'
  composers=['Nguyễn Đình Vũ'] | lyricists=['Nguyễn Đình Vũ'] | year=2021
  genres=['Trẻ', 'V-Pop'] | urls=['https://hopamviet.vn/chord/song/7-love/W8IU7UZ8.html', 'https://timbaihat.com/loi-bai-hat-7-love-nguyen-dinh-vu.html']

🧩 DEBUG MERGE #3: 'ai cho tôi tình yêu'
  composers=['Trúc Phương'] | lyricists=['Trúc Phương', 'Bảo Hân Bolero'] | year=2019
  genres=['Vàng', 'Trữ Tình'] | urls=['https://hopamviet.vn/chord/song/ai-cho-toi-tinh-yeu/W8IU0FWO.html', 'https://timbaihat.com/loi-bai-hat-ai-cho-toi-tinh-yeu-bao-

# Merge cách 2 (độ tương đồng lyric > 70%)

In [35]:
import pandas as pd
from rapidfuzz import fuzz
from tqdm import tqdm

def merge_unique_case_insensitive(a, b):
    merged = []
    seen = set()
    for lst in (a, b):
        if lst:
            for item in lst:
                if not isinstance(item, str):
                    continue
                key = item.lower().strip()
                if key not in seen and key != '':
                    seen.add(key)
                    merged.append(item.strip())
    return merged


def merge_rows_lyric(r1, r2, debug=False, debug_counter=[0], debug_limit=10):
    """Gộp 2 dòng dựa trên lyric giống nhau."""
    def is_empty(x):
        if x is None:
            return True
        if isinstance(x, (list, tuple, set)):
            return len(x) == 0
        if isinstance(x, str):
            return x.strip() == ''
        try:
            return pd.isna(x)
        except Exception:
            return False
    def pick(a, b):
        return a if not is_empty(a) else b

    merged = {
        'title': pick(r1.get('title'), r2.get('title')),
        'composers': merge_unique_case_insensitive(r1.get('composers'), r2.get('composers')),
        'lyricists': merge_unique_case_insensitive(r1.get('lyricists'), r2.get('lyricists')),
        'year': pick(r1.get('year'), r2.get('year')),
        'genres': merge_unique_case_insensitive(r1.get('genres'), r2.get('genres')),
        'lyrics': pick(r1.get('lyrics'), r2.get('lyrics')),
        'urls': merge_unique_case_insensitive(r1.get('urls'), r2.get('urls')),
        'source': pick(r1.get('source'), r2.get('source')),
        'note': pick(r1.get('note'), r2.get('note'))
    }

    if debug and debug_counter[0] < debug_limit:
        debug_counter[0] += 1
        print(f"\n🎵 DEBUG LYRIC MERGE #{debug_counter[0]}: '{merged['title']}'")
        print(f"  composers={merged['composers']} | lyricists={merged['lyricists']}")
        print(f"  genres={merged['genres']} | urls={merged['urls']}")
    return merged


def merge_by_lyrics(df, output_path='merged_final_lyric.csv', lyric_threshold=70, debug_merge=True, debug_limit=5):
    print(f"📘 Đọc dữ liệu merge trước đó ({len(df)} dòng)")

    debug_counter = [0]
    total_merge = 0
    merged_idx = set()
    merged_rows = []

    df = df.reset_index(drop=True)

    # === THÊM PROGRESS BAR ===
    for i in tqdm(range(len(df)), desc=f"🔍 So khớp lyric ≥{lyric_threshold}%"):
        if i in merged_idx:
            continue
        row_i = df.iloc[i]
        lyrics_i = str(row_i.get('lyrics', '')).strip()
        if not lyrics_i:
            merged_rows.append(row_i)
            continue

        matched = False
        for j in range(i + 1, len(df)):
            if j in merged_idx:
                continue
            row_j = df.iloc[j]
            lyrics_j = str(row_j.get('lyrics', '')).strip()
            if not lyrics_j:
                continue

            sim = fuzz.partial_ratio(lyrics_i, lyrics_j)
            if sim >= lyric_threshold:
                merged = merge_rows_lyric(
                    row_i, row_j,
                    debug=debug_merge,
                    debug_counter=debug_counter,
                    debug_limit=debug_limit
                )
                merged_rows.append(pd.Series(merged))
                merged_idx.add(j)
                matched = True
                total_merge += 1
                break

        if not matched:
            merged_rows.append(row_i)

    merged_df = pd.DataFrame(merged_rows)
    print(f"\n✅ Đã merge {total_merge} cặp bài hát dựa trên lyric ≥ {lyric_threshold}%")
    merged_df.to_csv(output_path, index=False)
    print(f"💾 Lưu kết quả: {output_path}")
    print(f"📊 Tổng số bài sau merge: {len(merged_df)}")
    return merged_df


# ==== DEMO CHẠY ====
df = pd.read_csv("merged_final.csv")
df['composers'] = df['composers'].apply(eval)
df['lyricists'] = df['lyricists'].apply(eval)
df['genres'] = df['genres'].apply(eval)
df['urls'] = df['urls'].apply(eval)

merged_df = merge_by_lyrics(
    df,
    output_path='merged_final_lyric.csv',
    lyric_threshold=70,
    debug_merge=True,
    debug_limit=3
)


📘 Đọc dữ liệu merge trước đó: merged_final.csv (129967 dòng)
⚡ Bắt đầu merge bằng multiprocessing (2 cores)


  1%|          | 6/1000 [15:03<41:35:54, 150.66s/it]Process ForkPoolWorker-7:



KeyboardInterrupt: 

Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)


Tối ưu cách Merge 2

In [ ]:
import pandas as pd
from rapidfuzz import fuzz
from tqdm import tqdm
import logging
from datetime import datetime
import os
import json
import hashlib

# ==================== LOGGING CONFIG ====================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)-8s | %(message)s",
    datefmt="%H:%M:%S",
    handlers=[
        logging.FileHandler("lyrics_merge.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# ==================== CHECKPOINT MANAGER ====================
class CheckpointManager:
    def __init__(self, checkpoint_dir='checkpoints_lyric'):
        self.checkpoint_dir = checkpoint_dir
        os.makedirs(checkpoint_dir, exist_ok=True)
        self.checkpoint_file = os.path.join(checkpoint_dir, 'merge_state.json')
        self.data_checkpoint = os.path.join(checkpoint_dir, 'merged_data.csv')
        
    def save(self, merged_rows, merged_idx, current_idx, total_merge):
        """Lưu checkpoint"""
        state = {
            'current_idx': current_idx,
            'merged_idx': list(merged_idx),
            'total_merge': total_merge,
            'timestamp': datetime.now().isoformat()
        }
        
        # Lưu state
        with open(self.checkpoint_file, 'w', encoding='utf-8') as f:
            json.dump(state, f, indent=2)
        
        # Lưu data
        temp_df = pd.DataFrame(merged_rows)
        temp_df.to_csv(self.data_checkpoint, index=False, encoding='utf-8-sig')
        
        logger.info(f"💾 Checkpoint saved at index {current_idx}")
    
    def load(self):
        """Tải checkpoint"""
        if not os.path.exists(self.checkpoint_file):
            return None
        
        with open(self.checkpoint_file, 'r', encoding='utf-8') as f:
            state = json.load(f)
        
        if os.path.exists(self.data_checkpoint):
            merged_rows = pd.read_csv(self.data_checkpoint, encoding='utf-8-sig').to_dict('records')
            logger.info(f"✅ Loaded checkpoint from index {state['current_idx']}")
            return state, merged_rows
        
        return None
    
    def clear(self):
        """Xóa checkpoint"""
        if os.path.exists(self.checkpoint_file):
            os.remove(self.checkpoint_file)
        if os.path.exists(self.data_checkpoint):
            os.remove(self.data_checkpoint)
        logger.info("🗑️ Checkpoint cleared")


# ==================== HELPER FUNCTIONS ====================
def merge_unique_case_insensitive(a, b):
    """Merge 2 list, loại bỏ trùng lặp (case-insensitive)"""
    merged = []
    seen = set()
    for lst in (a, b):
        if lst:
            for item in lst:
                if not isinstance(item, str):
                    continue
                key = item.lower().strip()
                if key not in seen and key != '':
                    seen.add(key)
                    merged.append(item.strip())
    return merged


def is_empty(x):
    """Kiểm tra giá trị rỗng"""
    if x is None:
        return True
    if isinstance(x, (list, tuple, set)):
        return len(x) == 0
    if isinstance(x, str):
        return x.strip() == ''
    try:
        return pd.isna(x)
    except Exception:
        return False


def pick(a, b):
    """Chọn giá trị không rỗng"""
    return a if not is_empty(a) else b


def merge_rows_lyric(r1, r2, debug=False):
    """Gộp 2 dòng dựa trên lyric giống nhau"""
    merged = {
        'title': pick(r1.get('title'), r2.get('title')),
        'composers': merge_unique_case_insensitive(r1.get('composers'), r2.get('composers')),
        'lyricists': merge_unique_case_insensitive(r1.get('lyricists'), r2.get('lyricists')),
        'year': pick(r1.get('year'), r2.get('year')),
        'genres': merge_unique_case_insensitive(r1.get('genres'), r2.get('genres')),
        'lyrics': pick(r1.get('lyrics'), r2.get('lyrics')),
        'urls': merge_unique_case_insensitive(r1.get('urls'), r2.get('urls')),
        'source': pick(r1.get('source'), r2.get('source')),
        'note': pick(r1.get('note'), r2.get('note'))
    }
    
    if debug:
        logger.debug(f"Merged: {merged['title']} | composers={len(merged['composers'])}, urls={len(merged['urls'])}")
    
    return merged


def get_lyrics_hash(lyrics):
    """Tạo hash từ lyrics để tăng tốc so sánh"""
    if not lyrics or not isinstance(lyrics, str):
        return None
    clean = lyrics.lower().strip()[:100]  # Chỉ lấy 100 ký tự đầu
    return hashlib.md5(clean.encode()).hexdigest()


# ==================== MAIN MERGE FUNCTION ====================
def merge_by_lyrics(df, 
                    output_path='merged_final_lyric.csv',
                    lyric_threshold=70,
                    checkpoint_interval=500,
                    use_hash_optimization=True,
                    resume_from_checkpoint=True):
    
    start_time = datetime.now()
    logger.info("="*60)
    logger.info("🎵 BẮT ĐẦU MERGE LYRICS")
    logger.info(f"📊 Dữ liệu: {len(df)} dòng")
    logger.info(f"🎯 Ngưỡng: {lyric_threshold}%")
    logger.info("="*60)
    
    # Khởi tạo checkpoint manager
    ckpt = CheckpointManager()
    
    # Thử load checkpoint
    checkpoint_data = None
    if resume_from_checkpoint:
        checkpoint_data = ckpt.load()
    
    df = df.reset_index(drop=True)
    
    # Khởi tạo biến
    if checkpoint_data:
        state, merged_rows = checkpoint_data
        merged_idx = set(state['merged_idx'])
        total_merge = state['total_merge']
        start_idx = state['current_idx'] + 1
        logger.info(f"🔄 Tiếp tục từ index {start_idx}, đã merge {total_merge} cặp")
    else:
        merged_rows = []
        merged_idx = set()
        total_merge = 0
        start_idx = 0
    
    # Tối ưu: Tạo hash map cho lyrics (nếu bật)
    lyrics_hash_map = {}
    if use_hash_optimization:
        logger.info("🔧 Tạo hash map để tối ưu tìm kiếm...")
        for i in range(len(df)):
            lyrics = str(df.iloc[i].get('lyrics', '')).strip()
            if lyrics:
                h = get_lyrics_hash(lyrics)
                if h:
                    if h not in lyrics_hash_map:
                        lyrics_hash_map[h] = []
                    lyrics_hash_map[h].append(i)
        logger.info(f"✅ Đã tạo {len(lyrics_hash_map)} hash groups")
    
    # Main loop với progress bar
    for i in tqdm(range(start_idx, len(df)), desc=f"🔍 Merge lyric ≥{lyric_threshold}%", initial=start_idx, total=len(df)):
        if i in merged_idx:
            continue
        
        row_i = df.iloc[i]
        lyrics_i = str(row_i.get('lyrics', '')).strip()
        
        if not lyrics_i:
            merged_rows.append(row_i)
            continue
        
        matched = False
        
        # Tối ưu: Chỉ so sánh với các bài có hash tương tự
        if use_hash_optimization:
            h = get_lyrics_hash(lyrics_i)
            candidates = lyrics_hash_map.get(h, [i])
            search_range = [j for j in candidates if j > i and j not in merged_idx]
        else:
            search_range = range(i + 1, len(df))
        
        for j in search_range:
            if j in merged_idx:
                continue
            
            row_j = df.iloc[j]
            lyrics_j = str(row_j.get('lyrics', '')).strip()
            
            if not lyrics_j:
                continue
            
            # Tối ưu: Kiểm tra độ dài trước khi so sánh
            len_diff = abs(len(lyrics_i) - len(lyrics_j)) / max(len(lyrics_i), len(lyrics_j))
            if len_diff > 0.5:  # Nếu chênh lệch >50% độ dài thì bỏ qua
                continue
            
            sim = fuzz.partial_ratio(lyrics_i, lyrics_j)
            
            if sim >= lyric_threshold:
                merged = merge_rows_lyric(row_i, row_j, debug=False)
                merged_rows.append(pd.Series(merged))
                merged_idx.add(j)
                matched = True
                total_merge += 1
                
                logger.debug(f"✓ Merged pair: {i} + {j} (sim={sim}%)")
                break
        
        if not matched:
            merged_rows.append(row_i)
        
        # Lưu checkpoint định kỳ
        if (i + 1) % checkpoint_interval == 0:
            ckpt.save(merged_rows, merged_idx, i, total_merge)
    
    # Tạo DataFrame kết quả
    merged_df = pd.DataFrame(merged_rows)
    
    # Lưu file cuối
    merged_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    
    # Clear checkpoint
    ckpt.clear()
    
    # Thống kê
    elapsed = datetime.now() - start_time
    logger.info("="*60)
    logger.info(f"✅ HOÀN TẤT MERGE LYRICS")
    logger.info(f"📊 Kết quả:")
    logger.info(f"   - Đã merge: {total_merge} cặp")
    logger.info(f"   - Trước: {len(df)} bài")
    logger.info(f"   - Sau: {len(merged_df)} bài")
    logger.info(f"   - Giảm: {len(df) - len(merged_df)} bài ({(len(df)-len(merged_df))/len(df)*100:.1f}%)")
    logger.info(f"⏱️  Thời gian: {elapsed}")
    logger.info(f"💾 File: {output_path}")
    logger.info("="*60)
    
    return merged_df


# ==================== DEMO CHẠY ====================
if __name__ == '__main__':
    logger.info("📂 Đọc file merged_final.csv...")
    df = pd.read_csv("merged_final.csv", encoding='utf-8-sig')
    
    # Parse list columns
    for col in ['composers', 'lyricists', 'genres', 'urls']:
        if col in df.columns:
            df[col] = df[col].apply(lambda x: eval(x) if isinstance(x, str) and x.startswith('[') else [])
    
    # Chạy merge
    merged_df = merge_by_lyrics(
        df,
        output_path='merged_final_lyric.csv',
        lyric_threshold=70,
        checkpoint_interval=500,  # Lưu mỗi 500 dòng
        use_hash_optimization=True,  # Bật tối ưu hash
        resume_from_checkpoint=True  # Tự động resume nếu có checkpoint
    )